In [97]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [98]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [99]:
torch.manual_seed(0)

train_dataset = torchvision.datasets.ImageFolder(root='train', transform=transform)

batch_size = 256

train_size = int(0.80 * len(train_dataset))
val_size = len(train_dataset) - train_size
trainset, valset = random_split(train_dataset, [train_size, val_size])

test_dataset = torchvision.datasets.ImageFolder(root='test', transform=transform)
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valloader = DataLoader(valset, batch_size=batch_size, shuffle=False)
testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [100]:
resnet = models.resnet152(weights=True)

for param in resnet.parameters():
    param.requires_grad = False

num_classes = 8

num_ftrs = resnet.fc.in_features

resnet.fc = nn.Sequential( 
    nn.Linear(num_ftrs, 1024),
    nn.ReLU(),
    nn.BatchNorm1d(1024),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.BatchNorm1d(512),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.BatchNorm1d(256),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.BatchNorm1d(128),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.BatchNorm1d(64),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.BatchNorm1d(32),
    nn.Linear(32, len(train_dataset.classes))
)
print(num_ftrs)
resnet.add_module('softmax', nn.Softmax(dim=1))

2048


In [101]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters(), lr=0.0005, weight_decay=0.05)
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=3, min_lr = 1e-6, verbose=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet.to(device)
print(device)

cuda:0


In [102]:
epochs = 6

for epoch in range(epochs):
    # Training phase
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        optimizer.zero_grad()

        outputs = resnet(inputs)
        # Calculate loss
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate training accuracy
        _, predicted_train = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted_train == labels).sum().item()

    train_loss = running_loss / len(trainloader)
    train_accuracy = 100 * correct_train / total_train
    
    # Validation phase
    correct_val = 0
    total_val = 0
    val_loss = 0.0
    with torch.no_grad():
        for data in valloader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = resnet(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted_val = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted_val == labels).sum().item()

    val_loss /= len(valloader)
    scheduler.step(val_loss)
    val_accuracy = 100 * correct_val / total_val
    
    print('Epoch %d, Training Loss: %.3f, Training Accuracy: %.2f%%, Validation Loss: %.3f, Validation Accuracy: %.2f%%' %
          (epoch + 1, train_loss, train_accuracy, val_loss, val_accuracy), "Current lr:", optimizer.param_groups[0]['lr'])
    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = resnet(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print("accuracy on testing", accuracy)

print('Finished Training')



Epoch 1, Training Loss: 1.535, Training Accuracy: 49.62%, Validation Loss: 1.107, Validation Accuracy: 87.59% Current lr: 0.0005
accuracy on testing 78.5
Epoch 2, Training Loss: 0.950, Training Accuracy: 86.50%, Validation Loss: 0.976, Validation Accuracy: 92.86% Current lr: 0.0005
accuracy on testing 86.0
Epoch 3, Training Loss: 0.738, Training Accuracy: 92.91%, Validation Loss: 0.860, Validation Accuracy: 94.74% Current lr: 0.0005
accuracy on testing 88.0
Epoch 4, Training Loss: 0.608, Training Accuracy: 95.40%, Validation Loss: 0.788, Validation Accuracy: 95.49% Current lr: 0.0005
accuracy on testing 87.0
Epoch 5, Training Loss: 0.528, Training Accuracy: 96.46%, Validation Loss: 0.715, Validation Accuracy: 96.99% Current lr: 0.0005
accuracy on testing 90.5
Epoch 6, Training Loss: 0.466, Training Accuracy: 97.36%, Validation Loss: 0.690, Validation Accuracy: 97.37% Current lr: 0.0005
accuracy on testing 91.0
Finished Training


In [103]:
total_percentage = 0
n_runs = 1
for i in range(n_runs):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = resnet(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    total_percentage += accuracy
print('Accuracy of the network on the %d test images: %.2f %%' % (total, total_percentage/n_runs))

Accuracy of the network on the 200 test images: 91.00 %


In [104]:
torch.save(resnet.state_dict(), 'model.pth')